<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/5_combined_model_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-06-15 18:15:37--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11733 (11K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]  11.46K  --.-KB/s    in 0s      

2022-06-15 18:15:37 (120 MB/s) - ‘utils.py’ saved [11733/11733]



In [3]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [4]:
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

!pip install -q tensorflow-io
import tensorflow_io as tfio

from utils import get_train_val_ds, get_class_weight

     |████████████████████████████████| 25.9 MB 1.2 MB/s 


In [5]:
BATCH_SIZE = 32
IMG_SIZE = (256, 256)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/bit/m-r50x1/1'

In [6]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [7]:
train_ds, _ = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE, img_size=IMG_SIZE)

Found 16580 files belonging to 16 classes.
Found 3508 files belonging to 16 classes.


In [8]:
classes, class_weight = get_class_weight()

In [9]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [10]:
target_shape = (32, 64, 1)

rgb_input = keras.Input(shape=IMG_SIZE+(3,))
hub_module = hub.KerasLayer(HUB_URL)

rgb_features = hub_module(rgb_input)
rgb_layer = keras.layers.Reshape(target_shape=target_shape)(rgb_features)

xyz_input = tfio.experimental.color.rgb_to_xyz(rgb_input)
xyz_features = hub_module(xyz_input)
xyz_layer = keras.layers.Reshape(target_shape=target_shape)(xyz_features)

hsv_input = tf.image.rgb_to_hsv(rgb_input)
hsv_features = hub_module(hsv_input)
hsv_layer = keras.layers.Reshape(target_shape=target_shape)(hsv_features)

block = keras.layers.Concatenate()([rgb_layer, xyz_layer, hsv_layer])

conv1 = keras.layers.Conv2D(filters=10,
                            kernel_size=(2,2),
                            strides=(1,2),
                            padding='same',
                            activation='relu')(block)

conv2 = keras.layers.Conv2D(filters=20,
                            kernel_size=(2,2),
                            strides=(2,2),
                            activation='relu')(conv1)

avg_pool = keras.layers.AveragePooling2D(pool_size=(2,2))(conv2)

conv3 = keras.layers.Conv2D(filters=20,
                            kernel_size=(1,1),
                            activation='relu')(avg_pool)

flatten = keras.layers.Flatten()(conv3)

output = keras.layers.Dense(N_CLASSES, activation='softmax')(flatten)

model = keras.models.Model(inputs=[rgb_input], outputs=[output])

In [11]:
model._name = 'combined-model'

In [12]:
metrics = [
            keras.metrics.AUC(name='auc', curve='PR', num_thresholds=100),
            'accuracy'
          ]

loss_fn = keras.losses.CategoricalCrossentropy()
lr_schedule =tf.keras.optimizers.schedules.ExponentialDecay(1e-2, 533, .9)

model.compile(loss=loss_fn,
            optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
            metrics=metrics)

In [13]:
early_stopping = keras.callbacks.EarlyStopping(
                                              monitor='auc', 
                                              verbose=1,
                                              patience=3,
                                              restore_best_weights=True,
                                              mode='max')

check_point_path = os.path.join('./logs/checkpoints', model._name+"-{epoch:02d}.h5")
check_point = keras.callbacks.ModelCheckpoint(
                                            filepath=check_point_path,
                                            monitor='auc',
                                            save_best_only=True,
                                            mode='max')
        
callbacks = [early_stopping, check_point]

In [14]:
history = model.fit(train_ds,
                    callbacks=callbacks,
                    epochs=20,
                    class_weight=class_weight)

Epoch 1/20
519/519 [==============================] - 2474s 5s/step - loss: 20.6054 - auc: 0.7934 - accuracy: 0.7113
Epoch 2/20
519/519 [==============================] - 433s 833ms/step - loss: 12.7492 - auc: 0.8807 - accuracy: 0.7982
Epoch 3/20
519/519 [==============================] - 434s 834ms/step - loss: 11.0154 - auc: 0.9003 - accuracy: 0.8239
Epoch 4/20
519/519 [==============================] - 434s 834ms/step - loss: 9.8305 - auc: 0.9148 - accuracy: 0.8359
Epoch 5/20
519/519 [==============================] - 434s 834ms/step - loss: 9.0306 - auc: 0.9232 - accuracy: 0.8486
Epoch 6/20
519/519 [==============================] - 434s 834ms/step - loss: 7.9557 - auc: 0.9357 - accuracy: 0.8672
Epoch 7/20
519/519 [==============================] - 434s 833ms/step - loss: 7.5632 - auc: 0.9412 - accuracy: 0.8767
Epoch 8/20
519/519 [==============================] - 434s 834ms/step - loss: 6.5960 - auc: 0.9497 - accuracy: 0.8878
Epoch 9/20
519/519 [==============================] - 4

In [15]:
!cp ./logs/checkpoints/combined-model-10.h5 ./drive/MyDrive/checkpoints/combined-model-10.1.h5
!cp ./logs/checkpoints/combined-model-15.h5 ./drive/MyDrive/checkpoints/combined-model-15.1.h5
!cp ./logs/checkpoints/combined-model-20.h5 ./drive/MyDrive/checkpoints/combined-model-20.1.h5

In [16]:
history = model.fit(train_ds,
                        callbacks=callbacks,
                        epochs=30,
                        initial_epoch=20,
                        class_weight=class_weight)

Epoch 21/30
519/519 [==============================] - 430s 823ms/step - loss: 2.0828 - auc: 0.9903 - accuracy: 0.9659
Epoch 22/30
519/519 [==============================] - 434s 833ms/step - loss: 1.9592 - auc: 0.9913 - accuracy: 0.9685
Epoch 23/30
519/519 [==============================] - 433s 833ms/step - loss: 1.8316 - auc: 0.9918 - accuracy: 0.9710
Epoch 24/30
519/519 [==============================] - 433s 833ms/step - loss: 1.7306 - auc: 0.9926 - accuracy: 0.9732
Epoch 25/30
519/519 [==============================] - 433s 832ms/step - loss: 1.6157 - auc: 0.9934 - accuracy: 0.9758
Epoch 26/30
519/519 [==============================] - 434s 834ms/step - loss: 1.5040 - auc: 0.9945 - accuracy: 0.9768
Epoch 27/30
519/519 [==============================] - 434s 834ms/step - loss: 1.4442 - auc: 0.9948 - accuracy: 0.9784
Epoch 28/30
519/519 [==============================] - 434s 835ms/step - loss: 1.3661 - auc: 0.9951 - accuracy: 0.9798
Epoch 29/30
519/519 [===========================

In [17]:
!cp ./logs/checkpoints/combined-model-25.h5 ./drive/MyDrive/checkpoints/combined-model-25.1.h5
!cp ./logs/checkpoints/combined-model-30.h5 ./drive/MyDrive/checkpoints/combined-model-30.1.h5